This notebook mimics the myc_notebook from pyetm v1 to demonstrate how functionalities remain the same, despite being dressed up differently.

**Multi-year charts**

The MYC function within the ETM allows users to view and edit multiple scenarios at once. This 
feature is useful to compare different scenarios or to built a transition path that contains 
scenarios with different end years, see https://myc.energytransitionmodel.com/


**Model Initialisation**

Where you used to initialize the 'model' with the `model.from_excel()` function, now we work with a 'scenario' or 'scenarios' (TBC - what will we call the multiple scenario object?).

You need to run the setup_notebook() command to get the API ready - this will also validate that you've set your API token correctly. You determine whether you connect with the beta, stable or live version of the ETM via the ETM_URL in the config.

In [ ]:
from example_helpers import setup_notebook
from pyetm.models import Scenario

setup_notebook()
# put a comment how to make the setup

Environment setup complete
  Using ETM API at     https://beta.engine.energytransitionmodel.com/api/v3
  Token loaded?        True
API connection ready


Then, you can create a scenario (or scenarios) with the Scenario.from_excel. For now this functionality is not implemented, so the example will continue by loading an individual scenario. Note that when multi-scenario functionality has been implemented, these same steps should work exactly the same way, but on the Scenarios object, rather than the Scenario object.

In [ ]:
scenario = Scenario.load(2690499)
scenario.to_dataframe()

,2690499
scenario,
end_year,2040
private,False
area_code,nl2019
template,2402157


In [ ]:
# Get scenario input parameters (incl. units)
inputs = scenario.inputs
# Give an explanation of available columns
print(inputs.to_dataframe(columns=['max', 'min', 'user']).head())

                                                    max  min user
input                                       unit                 
climate_relevant_co2_biomass_gas_future     %     100.0  0.0  0.0
climate_relevant_co2_biomass_gas_present    %     100.0  0.0  0.0
climate_relevant_co2_biomass_liquid_future  %     100.0  0.0  0.0
climate_relevant_co2_biomass_liquid_present %     100.0  0.0  0.0
climate_relevant_co2_biomass_solid_future   %     100.0  0.0  0.0


You will notice that in the original pyetm this call was:
```
inputs = model.get_parameters(exclude=True)
inputs.head()
```
Pyetm v2 is more developer friendly, which means that each of the 'objects' on the model (for example the inputs) are still python objects until you call **to_dataframe()** on them. This is one of the more significant changes - you will have to specify (in the notebooks) what format you want the data in. For the flows from Excel --> ETM and ETM --> Excel, this is all taken care of internally.

In [ ]:
# We also distinguish between inputs and user_values. Usually user values are more interesting:
user_values = scenario.user_values()
# Check if we can make this also to_dataframe
user_values

{'climate_relevant_co2_biomass_gas_future': 0.0,
 'climate_relevant_co2_biomass_gas_present': 0.0,
 'climate_relevant_co2_biomass_liquid_future': 0.0,
 'climate_relevant_co2_biomass_liquid_present': 0.0,
 'climate_relevant_co2_biomass_solid_future': 0.0,
 'climate_relevant_co2_biomass_solid_present': 0.0,
 'capacity_costs_energy_flexibility_flow_batteries_electricity': 0.0,
 'costs_bio_ethanol': 0.9482455192,
 'costs_biodiesel': 0.9588091368,
 'costs_biogas': 58.0,
 'costs_captured_biogenic_co2': 0.0,
 'costs_co2': 123.0,
 'costs_co2_free_allocation': 0.0,
 'costs_coal': 56.0,
 'costs_electricity_fallback_price': 3000.0,
 'costs_gas': 15.0,
 'costs_greengas': 61.0,
 'costs_heat_infra_indoors': 100.0,
 'costs_heat_infra_outdoors': 100.0,
 'costs_heat_network_storage_ht_steam_hot_water': 43.2,
 'costs_hydrogen': 30.0,
 'costs_hydrogen_transport_compressed_trucks': 50.0822647794,
 'costs_hydrogen_transport_pipelines': 1.5706418062,
 'costs_imported_ammonia': 43.2145152,
 'costs_imported_h

In [19]:
# Execute gqueries to fetch data from ETM (incl. units)
gqueries = scenario.results()
print(gqueries)

Empty DataFrame
Columns: []
Index: []


There are no default gqueries, you need to specify them yourself - hence the empty dataframe .

In [20]:
scenario.add_queries(["dashboard_emissions"])
gqueries = scenario.results()
print(gqueries)

                              future
gquery              unit            
dashboard_emissions factor -0.780545


In [ ]:
# Optional: Fetch hourly electricity price profiles from ETM
custom_curves = scenario.custom_curves_series()
for key in custom_curves: print(key)

#TODO: Discuss

Do you want a convenient way to only get the custom curves related to electricity price? Or are you more interested in the carrier curves? 

In [ ]:
# Optional: Fetch hourly carrier profiles from ETM
output_curves = scenario.get_output_curves(carrier_type='electricity')
# Also serval carriers at ones. One carrier per sheet, with carrier name as sheet name
output_curves

{'merit_order':       agriculture_chp_engine_biogas.output (MW)  \
 0                                           0.0   
 1                                           0.0   
 2                                           0.0   
 3                                           0.0   
 4                                           0.0   
 ...                                         ...   
 8755                                        0.0   
 8756                                        0.0   
 8757                                        0.0   
 8758                                        0.0   
 8759                                        0.0   
 
       agriculture_chp_engine_network_gas_dispatchable.output (MW)  \
 0                                                   0.0             
 1                                                   0.0             
 2                                                   0.0             
 3                                                   0.0             
 4       

As you can see, this returns a dictionary of dataframes, including the merit_order, electricity_price and residual_load dataframes. If you want a specific output, just request it:

In [ ]:
electricity_price = scenario.output_curve(curve_name='electricity_price')
# Also in it's own sheet
# Hydrogen price also in it's own sheet

# Nice to have: also make other downloads like sankey and energy_flows available
electricity_price

**Export scenarios to Excel**

Just like with the original implementation, everything in the scenario can be exported to an excel file. This is done via a "Scenario Packer" which organises the data usefully to display via excel.

This example is simplified in the sense that we just add everything in the scenario to the packer, but you can also add specific inputs, gqueries curves etc to a packer (see more examples in the scenario_to_excel notebook).

In [ ]:
from pyetm.models import ScenarioPacker

packer = ScenarioPacker()
packer.add(scenario, scenario2, scenario3)
packer.to_excel('folder/my_excel.xlsx')

# Specify explicitly what to export in to_excel
packer.to_excel('folder/my_excel.xlsx', include_curves=True)